In [2]:
import tensorflow as tf
import tensorflow.keras as keras
from keras.models import Sequential, Model
from keras.layers import Dense
from keras.optimizers import Adam
import pandas as pd
from sklearn.model_selection import train_test_split

In [8]:
dfX = pd.read_csv("summary/data_sd.csv")
dfE = pd.DataFrame(dfX.pop("sd_energy"))
dfdE = pd.DataFrame(dfX.pop("sd_denergy"))

In [14]:
dfX = (dfX - dfX.mean())/dfX.std()
dfE = (dfE - dfE.mean())/dfE.std()
dfdE = (dfdE - dfdE.mean())/dfdE.std()

X_train_full, X_test, e_train_full, e_test = train_test_split(dfX, dfE, test_size=0.2, random_state=42)
X_train, X_valid, e_train, e_valid = train_test_split(X_train_full, e_train_full, random_state=42)

In [22]:
modelE = Sequential([
    Dense(64, activation="relu", input_shape=X_train.shape[1:]),
    Dense(256, activation="relu"),
    Dense(128, activation="relu"),
    Dense(1)
])

modelE.compile(loss="mean_squared_error", optimizer=Adam(learning_rate=0.001))
history = modelE.fit(X_train, e_train, epochs=30, validation_data=(X_valid, e_valid))

Epoch 1/30
427/427 [==============================] - 1s 2ms/step - loss: 0.9858 - val_loss: 0.8878
Epoch 2/30
427/427 [==============================] - 1s 2ms/step - loss: 0.9858 - val_loss: 0.8878
Epoch 3/30
427/427 [==============================] - 1s 2ms/step - loss: 0.9859 - val_loss: 0.8880
Epoch 4/30
427/427 [==============================] - 1s 2ms/step - loss: 0.9859 - val_loss: 0.8876
Epoch 5/30
427/427 [==============================] - 1s 2ms/step - loss: 0.9859 - val_loss: 0.8878
Epoch 6/30
427/427 [==============================] - 1s 2ms/step - loss: 0.9859 - val_loss: 0.8881
Epoch 7/30
427/427 [==============================] - 1s 2ms/step - loss: 0.9858 - val_loss: 0.8880
Epoch 8/30
427/427 [==============================] - 1s 2ms/step - loss: 0.9859 - val_loss: 0.8874
Epoch 9/30
427/427 [==============================] - 1s 2ms/step - loss: 0.9858 - val_loss: 0.8885
Epoch 10/30
427/427 [==============================] - 1s 2ms/step - loss: 0.9859 - val_loss: 0.8877

In [23]:
modelE.save("modelo_E.h5")

In [24]:
X_train_full, X_test, de_train_full, de_test = train_test_split(dfX, dfdE, test_size=0.2, random_state=42)
X_train, X_valid, de_train, de_valid = train_test_split(X_train_full, de_train_full, random_state=42)

modeldE = Sequential([
    Dense(128, activation="relu", input_shape=X_train.shape[1:]),
    Dense(512, activation="relu"),
    Dense(256, activation="relu"),
    Dense(1)
])

modeldE.compile(loss="mean_squared_error", optimizer=Adam(learning_rate=0.001))
history = modelE.fit(X_train, de_train, epochs=30, validation_data=(X_valid, de_valid))

Epoch 1/30
427/427 [==============================] - 1s 2ms/step - loss: 0.9731 - val_loss: 0.9648
Epoch 2/30
427/427 [==============================] - 1s 2ms/step - loss: 0.9731 - val_loss: 0.9644
Epoch 3/30
427/427 [==============================] - 1s 2ms/step - loss: 0.9731 - val_loss: 0.9645
Epoch 4/30
427/427 [==============================] - 1s 2ms/step - loss: 0.9730 - val_loss: 0.9647
Epoch 5/30
427/427 [==============================] - 1s 2ms/step - loss: 0.9730 - val_loss: 0.9638
Epoch 6/30
427/427 [==============================] - 1s 2ms/step - loss: 0.9731 - val_loss: 0.9645
Epoch 7/30
427/427 [==============================] - 1s 2ms/step - loss: 0.9731 - val_loss: 0.9644
Epoch 8/30
427/427 [==============================] - 1s 2ms/step - loss: 0.9731 - val_loss: 0.9642
Epoch 9/30
427/427 [==============================] - 1s 2ms/step - loss: 0.9731 - val_loss: 0.9643
Epoch 10/30
427/427 [==============================] - 1s 2ms/step - loss: 0.9730 - val_loss: 0.9646

In [25]:
modeldE.save("modelo_dE.h5")